<a href="https://colab.research.google.com/github/Ruwai/DS-Unit-4-Sprint-3-Neural-Networks/blob/master/module3-Intro-to-Keras/LS_DS_433_Keras_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Use the Keras Library to build a Multi-Layer Perceptron Model on the Boston Housing dataset

- The Boston Housing dataset comes with the Keras library so use Keras to import it into your notebook. 
- Normalize the data (all features should have roughly the same scale)
- Import the type of model and layers that you will need from Keras.
- Instantiate a model object and use `model.add()` to add layers to your model
- Since this is a regression model you will have a single output node in the final layer.
- Use activation functions that are appropriate for this task
- Compile your model
- Fit your model and report its accuracy in terms of Mean Squared Error
- Use the history object that is returned from model.fit to make graphs of the model's loss or train/validation accuracies by epoch. 
- Run this same data through a linear regression model. Which achieves higher accuracy?
- Do a little bit of feature engineering and see how that affects your neural network model. (you will need to change your model to accept more inputs)
- After feature engineering, which model sees a greater accuracy boost due to the new features?

In [21]:
##### Your Code Here #####

import tensorflow
tensorflow.__version__

'1.13.1'

In [0]:
import keras
from keras.datasets import boston_housing
from keras.models import Sequential
from keras.layers import Dense, Dropout  # Stretch - use dropout!
import numpy as np
# fix random seed for reproducibility
np.random.seed(42)

In [0]:
df = boston_housing.load_data()

In [0]:
epochs = 1000

(X_train,y_train),(X_test,y_test) = boston_housing.load_data()

In [52]:
X_train[0].shape

(13,)

In [53]:
X_train[0]

array([  1.23247,   0.     ,   8.14   ,   0.     ,   0.538  ,   6.142  ,
        91.7    ,   3.9769 ,   4.     , 307.     ,  21.     , 396.9    ,
        18.72   ])

In [54]:
y_train.shape

(404,)

In [55]:
y_train

array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6, 14.4, 12.1,
       17.9, 23.1, 19.9, 15.7,  8.8, 50. , 22.5, 24.1, 27.5, 10.9, 30.8,
       32.9, 24. , 18.5, 13.3, 22.9, 34.7, 16.6, 17.5, 22.3, 16.1, 14.9,
       23.1, 34.9, 25. , 13.9, 13.1, 20.4, 20. , 15.2, 24.7, 22.2, 16.7,
       12.7, 15.6, 18.4, 21. , 30.1, 15.1, 18.7,  9.6, 31.5, 24.8, 19.1,
       22. , 14.5, 11. , 32. , 29.4, 20.3, 24.4, 14.6, 19.5, 14.1, 14.3,
       15.6, 10.5,  6.3, 19.3, 19.3, 13.4, 36.4, 17.8, 13.5, 16.5,  8.3,
       14.3, 16. , 13.4, 28.6, 43.5, 20.2, 22. , 23. , 20.7, 12.5, 48.5,
       14.6, 13.4, 23.7, 50. , 21.7, 39.8, 38.7, 22.2, 34.9, 22.5, 31.1,
       28.7, 46. , 41.7, 21. , 26.6, 15. , 24.4, 13.3, 21.2, 11.7, 21.7,
       19.4, 50. , 22.8, 19.7, 24.7, 36.2, 14.2, 18.9, 18.3, 20.6, 24.6,
       18.2,  8.7, 44. , 10.4, 13.2, 21.2, 37. , 30.7, 22.9, 20. , 19.3,
       31.7, 32. , 23.1, 18.8, 10.9, 50. , 19.6,  5. , 14.4, 19.8, 13.8,
       19.6, 23.9, 24.5, 25. , 19.9, 17.2, 24.6, 13

In [56]:
X_train.shape

(404, 13)

In [0]:
X_train = keras.utils.normalize(X_train, axis=-1, order=2)
X_test = keras.utils.normalize(X_test, axis=-1, order=2)

In [58]:
X_test[0]

array([0.02675675, 0.        , 0.02677953, 0.        , 0.0010046 ,
       0.00951931, 0.14795322, 0.0027145 , 0.03550877, 0.98536841,
       0.02988655, 0.04031725, 0.04298041])

In [59]:
y_train[0:20]

array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6, 14.4, 12.1,
       17.9, 23.1, 19.9, 15.7,  8.8, 50. , 22.5, 24.1, 27.5])

In [62]:
model = Sequential()

model.add(Dense(13, input_dim=13, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(13, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='linear'))

model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['mean_squared_error'])
model.compile(optimizer='rmsprop', 
              loss='mean_squared_error', 
              metrics=['mean_squared_error'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 13)                182       
_________________________________________________________________
dropout_5 (Dropout)          (None, 13)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 13)                182       
_________________________________________________________________
dropout_6 (Dropout)          (None, 13)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 14        
Total params: 378
Trainable params: 378
Non-trainable params: 0
_________________________________________________________________


In [64]:
model.fit(X_train, y_train, epochs=epochs, validation_split=.1)
test_scores = model.evaluate(X_test, y_test)

print(f'After {epochs} epochs: ', f'{model.metrics_names[0]}: {test_scores[1]}')

Train on 363 samples, validate on 41 samples
Epoch 1/1000
363/363 [==============================] - 0s 642us/step - loss: 584.6028 - mean_squared_error: 584.6028 - val_loss: 492.5116 - val_mean_squared_error: 492.5116
Epoch 2/1000
363/363 [==============================] - 0s 46us/step - loss: 578.5777 - mean_squared_error: 578.5777 - val_loss: 487.3837 - val_mean_squared_error: 487.3837
Epoch 3/1000
363/363 [==============================] - 0s 48us/step - loss: 573.2931 - mean_squared_error: 573.2931 - val_loss: 481.9057 - val_mean_squared_error: 481.9057
Epoch 4/1000
363/363 [==============================] - 0s 47us/step - loss: 567.6832 - mean_squared_error: 567.6832 - val_loss: 475.9875 - val_mean_squared_error: 475.9875
Epoch 5/1000
363/363 [==============================] - 0s 49us/step - loss: 561.1107 - mean_squared_error: 561.1107 - val_loss: 469.5503 - val_mean_squared_error: 469.5503
Epoch 6/1000
363/363 [==============================] - 0s 48us/step - loss: 553.3391 - m

## Use the Keras Library to build an image recognition network using the Fashion-MNIST dataset (also comes with keras)

- Load and preprocess the image data similar to how we preprocessed the MNIST data in class.
- Make sure to one-hot encode your category labels
- Make sure to have your final layer have as many nodes as the number of classes that you want to predict.
- Try different hyperparameters. What is the highest accuracy that you are able to achieve.
- Use the history object that is returned from model.fit to make graphs of the model's loss or train/validation accuracies by epoch. 
- Remember that neural networks fall prey to randomness so you may need to run your model multiple times (or use Cross Validation) in order to tell if a change to a hyperparameter is truly producing better results.

In [0]:
##### Your Code Here #####
from keras.datasets import fashion_mnist

In [0]:
batch_size = 64
num_classes = 10
epochs = 100

In [68]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

X_train = X_train.reshape(60000,784)
X_test = X_test.reshape(10000,784)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

X_train /= 255
X_test /= 255

print(X_train.shape[0], 'Train Samples')
print(X_test.shape[0], 'Test Samples')

4423680/4422102 [==============================] - 1s 0us/step
60000 Train Samples
10000 Test Samples


In [69]:
model = Sequential()
model.add(Dense(32,activation='relu', input_shape=(784,)))
model.add(Dense(32,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='mean_squared_error',
              metrics=['mean_squared_error'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 32)                25120     
_________________________________________________________________
dense_12 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_13 (Dense)             (None, 10)                330       
Total params: 26,506
Trainable params: 26,506
Non-trainable params: 0
_________________________________________________________________


In [0]:
#https://github.com/keras-team/keras/blob/master/examples/mnist_dataset_api.py

import numpy as np
import os
import tempfile

import keras
from keras import backend as K
from keras import layers
from keras.datasets import fashion_mnist

import tensorflow as tf

def cnn_layers(inputs):
    X = layers.Conv2D(32, (3, 3),
                      activation='relu', padding='valid')(inputs)
    X = layers.MaxPooling2D(pool_size=(2, 2))(X)
    X = layers.Conv2D(64, (3, 3), activation='relu')(X)
    X = layers.MaxPooling2D(pool_size=(2, 2))(X)
    X = layers.Flatten()(X)
    X = layers.Dense(512, activation='relu')(X)
    X = layers.Dropout(0.5)(X)
    predictions = layers.Dense(num_classes,
                               activation='softmax',
                               name='X_train_out')(X)
    return predictions


batch_size = 128
buffer_size = 10000
steps_per_epoch = int(np.ceil(60000 / float(batch_size)))  # = 469
epochs = 5
num_classes = 10

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.astype(np.float32) / 255
X_train = np.expand_dims(X_train, -1)
y_train = tf.one_hot(y_train, num_classes)

# Create the dataset and its associated one-shot iterator.
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
dataset = dataset.repeat()
dataset = dataset.shuffle(buffer_size)
dataset = dataset.batch(batch_size)
iterator = dataset.make_one_shot_iterator()

# Model creation using tensors from the get_next() graph node.
inputs, targets = iterator.get_next()
model_input = layers.Input(tensor=inputs)
model_output = cnn_layers(model_input)
train_model = keras.models.Model(inputs=model_input, outputs=model_output)

train_model.compile(optimizer=keras.optimizers.RMSprop(lr=2e-3, decay=1e-5),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'],
                    target_tensors=[targets])
train_model.summary()

train_model.fit(epochs=epochs,
                steps_per_epoch=steps_per_epoch)

# Save the model weights.
weight = os.path.join(tempfile.gettempdir(), 'saved_weight.h5')
train_model.save_weights(weight)

# Clean up the TF session.
K.clear_session()

# Second session to test loading trained model without tensors.
X_test = X_test.astype(np.float32)
X_test = np.eXpand_dims(X_test, -1)

X_test_inp = layers.Input(shape=X_test.shape[1:])
test_out = cnn_layers(X_test_inp)
test_model = keras.models.Model(inputs=X_test_inp, outputs=test_out)

test_model.load_weights(weight_path)
test_model.compile(optimizer='rmsprop',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])
test_model.summary()

loss, acc = test_model.evaluate(X_test, y_test, num_classes)
print('\nTest accuracy: {0}'.format(acc))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 512)               819712    
__________

## Stretch Goals:

- Use Hyperparameter Tuning to make the accuracy of your models as high as possible. (error as low as possible)
- Use Cross Validation techniques to get more consistent results with your model.
- Use GridSearchCV to try different combinations of hyperparameters. 
- Start looking into other types of Keras layers for CNNs and RNNs maybe try and build a CNN model for fashion-MNIST to see how the results compare.